In [1]:
import os
from urllib import parse
import psycopg2
import numpy as np
import pandas as pd
import requests
import json
import datetime

/Users/stephaniewillis/.virtualenvs/ski/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:

parse.uses_netloc.append("postgres")
url = parse.urlparse(os.environ.get("DATABASE_URL"))

conn = psycopg2.connect(
    database=url.path[1:],
    user=url.username,
    password=url.password,
    host=url.hostname,
    port=url.port)


In [3]:
cursor = conn.cursor()

In [4]:
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cursor.fetchall())

[('temperatures',), ('forecasts',), ('current',)]


In [22]:
cursor=conn.cursor()
cursor.execute("CREATE TABLE temperatures( \
                resort TEXT, \
                time TIMESTAMP, \
                measurement REAL, \
                PRIMARY KEY(resort, time));")

In [160]:
conn.rollback()

In [90]:
conn.commit()

In [30]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
)

2018-02-17 14:11:09


In [99]:
base_str = "INSERT INTO temperatures (resort, time, measurement) VALUES ( \
            'Sutton',  '%s',  %1.2f)"

for i in range(10):
    timestamp = (datetime.datetime.now()+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    measurement = np.random.randn()

    cursor.execute(base_str%(timestamp, measurement))

    
conn.commit()

In [92]:
cursor=conn.cursor()
cursor.execute("SELECT * from temperatures;")


In [93]:
cursor.fetchone()

('jay', datetime.datetime(2018, 2, 17, 14, 28, 2), -1.12)

In [7]:
df = pd.read_sql('SELECT * from current;', conn)

In [8]:
df

,location,timestamp,rain_mm,snow_mm,temp_c_max,temp_c_min,windspeed_mph
0,Jay Peak,2018-02-17 21:10:00,0.0,0.0,-3.2,-3.7,7.0
1,Mt Sutton,2018-02-17 21:10:00,0.0,0.0,-3.2,-3.7,7.0
2,Mt Tremblant,2018-02-17 21:10:00,0.0,0.0,-6.0,-7.0,4.0
3,St Anne,2018-02-17 21:10:00,0.0,0.0,-10.6,-10.6,3.0
4,Le Massif,2018-02-17 21:10:00,0.0,0.0,-5.7,-5.7,6.0


In [100]:
baseurl = 'https://api.weatherunlocked.com/'

In [123]:
r=requests.get('http://api.weatherunlocked.com/api/current/51.50,-0.12?app_id=6765a93c&app_key=51b7b6cb5cf11b77b78d84d8f5c44845')

In [124]:
r.text

'{"lat":51.5,"lon":0.05,"alt_m":5.0,"alt_ft":16.4,"wx_desc":"Partly cloudy","wx_code":1,"wx_icon":"PartlyCloudyNight.gif","temp_c":6.0,"temp_f":42.8,"feelslike_c":4.59,"feelslike_f":40.26,"humid_pct":75.0,"windspd_mph":4.35,"windspd_kmh":7.0,"windspd_kts":3.78,"windspd_ms":1.94,"winddir_deg":220.0,"winddir_compass":"SW","cloudtotal_pct":50.0,"vis_km":10.0,"vis_mi":6.21,"vis_desc":null,"slp_mb":1023.0,"slp_in":30.29,"dewpoint_c":1.93,"dewpoint_f":35.47}'

In [15]:
loc_dict = {'Jay Peak':(44.9649,-72.4602), 
            'Mt Sutton':(45.1047, -72.5618),
            'Mt Tremblant': (46.1185,-74.5962),
            'St Anne': (47.0754,-70.9049),
            'Le Massif': (47.2848,-70.5697),
           }

In [110]:
r=requests.get('http://api.weatherunlocked.com/api/forecast/%1.2f,%1.2f?app_id=6765a93c&app_key=51b7b6cb5cf11b77b78d84d8f5c44845'%(loc_dict['Mt Sutton']))

In [111]:
result = json.loads(r.text)

In [132]:
result['Days'][0]['date']

'17/02/2018'

In [109]:
result

{'alt_ft': 498.69,
 'alt_m': 152.0,
 'cloudtotal_pct': 50.0,
 'dewpoint_c': -12.66,
 'dewpoint_f': 9.21,
 'feelslike_c': -5.15,
 'feelslike_f': 22.72,
 'humid_pct': 43.0,
 'lat': 45.05,
 'lon': -72.83,
 'slp_in': 30.18,
 'slp_mb': 1019.0,
 'temp_c': -1.8,
 'temp_f': 28.76,
 'vis_desc': None,
 'vis_km': 14.0,
 'vis_mi': 8.7,
 'winddir_compass': 'SSE',
 'winddir_deg': 160.0,
 'windspd_kmh': 9.0,
 'windspd_kts': 4.86,
 'windspd_mph': 5.59,
 'windspd_ms': 2.5,
 'wx_code': 1,
 'wx_desc': 'Partly cloudy',
 'wx_icon': 'PartlyCloudyNight.gif'}

In [193]:
def get_current_weather(loc_dict):
    
    data_dict = {}
    for location in loc_dict.keys():
        
        r = requests.get('http://api.weatherunlocked.com/api/forecast/%1.2f,%1.2f?app_id=6765a93c&app_key=51b7b6cb5cf11b77b78d84d8f5c44845'%(loc_dict[location]))
        
        result = json.loads(r.text)
        
        data_dict[location] = {'Windspeed_Mph':result['Days'][0]['windspd_max_mph'],
                               'Rain_mm': result['Days'][0]['rain_total_mm'],
                               'Snow_mm': result['Days'][0]['snow_total_mm'],
                               'Temp_C_Max': result['Days'][0]['temp_max_c'],
                               'Temp_C_Min': result['Days'][0]['temp_min_c'],
                               'Timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
                              } 
    
    return data_dict

def get_forecast_weather(loc_dict):
    
    data_dict = {}
    
    for location in loc_dict.keys():

        r = requests.get('http://api.weatherunlocked.com/api/forecast/%1.2f,%1.2f?app_id=6765a93c&app_key=51b7b6cb5cf11b77b78d84d8f5c44845'%(loc_dict[location]))
        
        result = json.loads(r.text)
        tomorrow = result['Days'][1]['Timeframes']
        
        data_dict[location] ={}
        for i in range(8):
            timestamp = (datetime.datetime.strptime(result['Days'][1]['date'],'%d/%m/%Y')
                         +datetime.timedelta(hours=int(tomorrow[i]['time'])/100)).strftime('%Y-%m-%d %H:%M')
            data_dict[location][timestamp]= {'Windspeed_Mph':tomorrow[i]['windspd_mph'],
                                                       'Rain_mm': tomorrow[i]['rain_mm'],
                                                       'Snow_mm': tomorrow[i]['snow_mm'],
                                                       'Temp_C_Max': tomorrow[i]['temp_c'],
                                                       'Temp_C_Min': tomorrow[i]['temp_c'],

                              } 
        
    
    return data_dict
    
def add_row(data, table_name):
    base_str = "INSERT INTO temperatures (resort, time, measurement) VALUES ( \
            'Sutton',  '%s',  %1.2f)"
    
    columns = data.keys()
    values = [data[column] for column in columns]

    insert_statement = 'INSERT into ' + table_name + ' (%s) values %s'

    cursor=conn.cursor()
    cursor.execute(insert_statement, (AsIs(','.join(columns)), tuple(values)))
    
    
def add_forecasts(forecast_dict):
    
    for location in forecast_dict.keys():

        for timestamp in forecast_dict[location]:
            data = forecast_dict[location][timestamp]
            data['Timestamp'] = timestamp
            data['Location'] = location

            add_row(data, 'forecasts')
            
def add_current(current_dict):
    
    for location in forecast_dict.keys():

        data = current_dict[location]
        data['Location'] = location

        add_row(data, 'current')

In [143]:
forecast_dict = get_forecast_weather(loc_dict)
forecast_dict

{'Jay Peak': {'2018-02-18 02:00': {'Rain mm': 0.0,
   'Snow mm': 0.0,
   'Temp C Max': -5.2,
   'Temp C Min': -5.2,
   'Windspeed mph': 6.0},
  '2018-02-18 05:00': {'Rain mm': 0.0,
   'Snow mm': 1.0,
   'Temp C Max': -5.7,
   'Temp C Min': -5.7,
   'Windspeed mph': 4.0},
  '2018-02-18 08:00': {'Rain mm': 0.0,
   'Snow mm': 0.0,
   'Temp C Max': -5.2,
   'Temp C Min': -5.2,
   'Windspeed mph': 7.0},
  '2018-02-18 11:00': {'Rain mm': 0.0,
   'Snow mm': 0.0,
   'Temp C Max': -3.8,
   'Temp C Min': -3.8,
   'Windspeed mph': 10.0},
  '2018-02-18 14:00': {'Rain mm': 0.0,
   'Snow mm': 0.0,
   'Temp C Max': -2.9,
   'Temp C Min': -2.9,
   'Windspeed mph': 12.0},
  '2018-02-18 17:00': {'Rain mm': 0.0,
   'Snow mm': 0.0,
   'Temp C Max': -3.2,
   'Temp C Min': -3.2,
   'Windspeed mph': 9.0},
  '2018-02-18 20:00': {'Rain mm': 0.0,
   'Snow mm': 0.0,
   'Temp C Max': -3.4,
   'Temp C Min': -3.4,
   'Windspeed mph': 4.0},
  '2018-02-18 23:00': {'Rain mm': 0.0,
   'Snow mm': 0.0,
   'Temp C Max': -

In [182]:
curr_dict = get_current_weather(loc_dict)

In [183]:
curr_dict

{'Jay Peak': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -3.2,
  'Temp_C_Min': -3.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 7.0},
 'Le Massif': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -5.7,
  'Temp_C_Min': -5.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 6.0},
 'Mt Sutton': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -3.2,
  'Temp_C_Min': -3.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 7.0},
 'Mt Tremblant': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -6.0,
  'Temp_C_Min': -7.0,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 4.0},
 'St Anne': {'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -10.6,
  'Temp_C_Min': -10.6,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 3.0}}

In [172]:
r=requests.get('http://api.openweathermap.org/data/2.5/weather?lat=35&lon=139&APPID=4061abd841220f4e41b4b7e7429a79f')

In [173]:
r.text

'{"cod":401, "message": "Invalid API key. Please see http://openweathermap.org/faq#error401 for more info."}'

In [174]:
r = requests.get('http://api.openweathermap.org/data/2.5/forecast?id=524901&APPID=4061abd841220f4e41b4b7e7429a79f')

In [175]:
r.text

'{"cod":401, "message": "Invalid API key. Please see http://openweathermap.org/faq#error401 for more info."}'

In [163]:
cursor=conn.cursor()
cursor.execute("CREATE TABLE forecasts( \
                Location TEXT, \
                Timestamp TIMESTAMP, \
                Rain_mm REAL, \
                Snow_mm REAL, \
                Temp_C_Max REAL, \
                Temp_C_Min REAL, \
                Windspeed_Mph REAL, \
                PRIMARY KEY(Location, Timestamp));")

conn.commit()

In [184]:
cursor=conn.cursor()
cursor.execute("CREATE TABLE current( \
                Location TEXT, \
                Timestamp TIMESTAMP, \
                Rain_mm REAL, \
                Snow_mm REAL, \
                Temp_C_Max REAL, \
                Temp_C_Min REAL, \
                Windspeed_Mph REAL, \
                PRIMARY KEY(Location, Timestamp));")

conn.commit()

In [169]:
conn.commit()

In [195]:
conn.rollback()


In [ ]:
curr_dict

In [191]:
curr_dict

{'Jay Peak': {'Location': 'Jay Peak',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -3.2,
  'Temp_C_Min': -3.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 7.0},
 'Le Massif': {'Location': 'Le Massif',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -5.7,
  'Temp_C_Min': -5.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 6.0},
 'Mt Sutton': {'Location': 'Mt Sutton',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -3.2,
  'Temp_C_Min': -3.7,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 7.0},
 'Mt Tremblant': {'Location': 'Mt Tremblant',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -6.0,
  'Temp_C_Min': -7.0,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 4.0},
 'St Anne': {'Location': 'St Anne',
  'Rain_mm': 0.0,
  'Snow_mm': 0.0,
  'Temp_C_Max': -10.6,
  'Temp_C_Min': -10.6,
  'Timestamp': '2018-02-17 21:10',
  'Windspeed_Mph': 3.0}}

In [196]:
add_current(curr_dict)

In [197]:
conn.commit()